In [ ]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel
from typing import List
import numpy as np
from itertools import combinations
import json
import statistics as st

In [ ]:
class SequenceList(BaseModel):
    sequence: List
parser = PydanticOutputParser(pydantic_object=SequenceList)
# print(f"Parser info: {parser.get_format_instructions()}")

api_key_list = [
  # Put your verifier model keys here
]

model_list = [
  # Put your verifier models here
]

def Extract_List(input, api_key_list, model_list):
  track = 0
  output =[]
  count =0
  mc = 0

  for i in input:
    track += 1
    model = ChatGroq(
      model=model_list[mc],
      api_key=api_key_list[count],
      temperature=0,
      timeout=None,
      max_retries=2,
    )


    count +=1
    if count==4:
      count =0
      mc+=1
    if mc==3:
      mc = 0

    messages = [
        (
            "system",
            f"Extract the list of sequence from input and return in this exact JSON format:\n"
            f"{parser.get_format_instructions()}\n"
            f"Example output: {{\"sequence\": [1, 2, 3]}}"
        ),
        ("human", f"Input is :: {i}"),
    ]

    ai_msg = model.invoke(messages)
    parsed = parser.parse(ai_msg.content)
    output.append(parsed.sequence)
    if track%10 == 0:
      print(f"Track: {track}")
  return output


# input = ["the sequenc that describes the scenerio is [1 3 2 4] which seems correct based on what you have given",
#          "the seq is [4.3.2.1] as far as I can see, it might be wrong but that's all I know",
#          "[4,2,1,3 goonan work"]
# output = Extract_List(input, api_key_list, model_list)
# print(output)

In [ ]:
def get_GoldenSeq(arr):
  arr = np.array(arr)
  output =[]
  output.append(np.argwhere(arr==0)[0][0])
  output.append(np.argwhere(arr==1)[0][0])
  output.append(np.argwhere(arr==2)[0][0])
  output.append(np.argwhere(arr==3)[0][0])

  return (np.array(output)+1).tolist()

def pairwise_accuracy(predicted, true_order):
    true_pos = {x: i for i, x in enumerate(true_order)}

    total_pairs = 0
    correct_pairs = 0
    for a, b in combinations(predicted, 2):
      try:
          if (true_pos[a] < true_pos[b] and predicted.index(a) < predicted.index(b)) or \
            (true_pos[a] > true_pos[b] and predicted.index(a) > predicted.index(b)):
              correct_pairs += 1
          total_pairs += 1
      except:
        continue
    # print(correct_pairs, total_pairs)
    return correct_pairs / total_pairs if total_pairs > 0 else 0.0


In [318]:
overall_score = []

In [ ]:
fs = [
    # Output files from the Models
]

In [ ]:
file_name = fs[4]
print(file_name)
with open(file_name, 'r',encoding = 'utf-8') as f:
  jf = json.load(f)
jf[0][2]

In [ ]:
tag = file_name.split('/')[2].split('.')[0]
if "Dhruv" in tag:
  tag = "Sports"
elif "Jeet" in tag:
  tag = "Culture"
elif "Rahul" in tag:
  tag = "Nature"
elif "Sai" in tag:
  tag = "Daily Routine"
elif "Pranav" in tag:
  tag = "Historical Events"

print(tag)

In [352]:
input = []
real_seq = []
for i in range(len(jf)):
  input.append(jf[i][1])
  # print(f"Input: {jf[i][2]}")
  real_seq.append(get_GoldenSeq(jf[i][2]))
  # print(f"Real: {real_seq}")

In [ ]:
model_seq = Extract_List(input, api_key_list, model_list)
print(f"real seq: {len(real_seq)} \n model_seq : {len(model_seq)}")

In [ ]:
mat_input = []
for i in range(len(real_seq)):
  mat_input.append((real_seq[i],model_seq[i]))

print(len(mat_input))

In [ ]:
score = []

for i in mat_input:
  accuracy = pairwise_accuracy(i[1],i[0])
  score.append(accuracy)
  overall_score.append(accuracy)

print(f"Domain: {tag}")
print(f"Total: {len(score)}")
print(f"Mean: {st.mean(score)}")
print(f"Median: {st.median(score)}")

In [ ]:
print("Overall Scores")
print(f"Total: {len(overall_score)}")
print(f"Mean: {st.mean(overall_score)}")
print(f"Median: {st.median(overall_score)}")